In [9]:
!pip install -qqq datasets==3.2.0 transformers==4.47.1 trl==0.14.0 peft==0.14.0 accelerate==1.2.1 bitsandbytes==0.45.2 wandb==0.19.7 --progress-bar off


In [3]:
!pip install -qqq flash-attn --no-build-isolation --progress-bar off

In [7]:
!pip install torch==2.5.1+cu121 --index-url https://download.pytorch.org/whl/cu121
!pip install torchvision==0.23.0 torchaudio==2.5.1



Looking in indexes: https://download.pytorch.org/whl/cu121

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
ERROR: Could not find a version that satisfies the requirement torchaudio==2.5.1 (from versions: 2.6.0, 2.7.0, 2.7.1, 2.8.0)

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
ERROR: No matching distribution found for torchaudio==2.5.1


In [10]:
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
from transformers import AutoModelForCausalLM, AutoTokenizer
from trl import GRPOConfig, GRPOTrainer
import wandb
wandb.login()

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

  ········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /home/tastyvibrator/.netrc
wandb: Currently logged in as: prxshetty (prxshetty-rochester-institute-of-technology) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [11]:
dataset = load_dataset("mlabonne/smoltldr")
print(dataset)

README.md:   0%|          | 0.00/981 [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/1.44M [00:00<?, ?B/s]

data/validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/200 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/200 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 2000
    })
    validation: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 200
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 200
    })
})


In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

model_id = "HuggingFaceTB/SmolLM-135M-Instruct"
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
)

tokenizer = AutoTokenizer.from_pretrained(model_id)


config.json:   0%|          | 0.00/723 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/269M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/565 [00:00<?, ?B/s]

In [13]:
lora_config = LoraConfig(
    task_type = "CAUSAL_LM",
    r = 16,
    lora_alpha = 32,
    target_modules = "all-linear",
)
model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

trainable params: 4,884,480 || all params: 139,399,488 || trainable%: 3.5039
None


In [14]:
ideal_length = 50
def reward_len(completions, **kwargs):
    return [-abs(ideal_length - len(completion)) for completion in completions]

In [15]:
training_args = GRPOConfig(
    output_dir="GRPO",
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    max_prompt_length=512,
    max_completion_length=96,
    num_generations=4,
    optim="adamw_8bit",
    num_train_epochs=1,
    bf16=True,
    report_to=["wandb"],
    remove_unused_columns=False,
    logging_steps=1,
)

In [16]:
trainer = GRPOTrainer(
    model=model,
    reward_funcs=[reward_len],
    args=training_args,
    train_dataset=dataset["train"],
)
wandb.init(project="GRPO")
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss
1,0.000000
2,0.000000
3,0.000000
4,0.000000
5,0.000000
6,0.000000
7,0.000000
8,0.000000
9,0.000000
10,0.000000


TrainOutput(global_step=250, training_loss=0.009838598390109837, metrics={'train_runtime': 7566.7669, 'train_samples_per_second': 0.264, 'train_steps_per_second': 0.033, 'total_flos': 0.0, 'train_loss': 0.009838598390109837})

In [23]:
tokenizer.push_to_hub("prxshetty/SmolGRPO-135M")

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/prxshetty/SmolGRPO-135M/commit/d3a2acc318533f35e527c19341565d0471cd2fd2', commit_message='Upload tokenizer', commit_description='', oid='d3a2acc318533f35e527c19341565d0471cd2fd2', pr_url=None, repo_url=RepoUrl('https://huggingface.co/prxshetty/SmolGRPO-135M', endpoint='https://huggingface.co', repo_type='model', repo_id='prxshetty/SmolGRPO-135M'), pr_revision=None, pr_num=None)

In [17]:
merged_model = trainer.model.merge_and_unload()
merged_model.push_to_hub(
    "SmolGRPO-135M", private=False, tags=["GRPO"]
)



Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  /tmp/tmp6zxdguwv/model.safetensors    :   0%|          | 12.0kB /  538MB            

CommitInfo(commit_url='https://huggingface.co/prxshetty/SmolGRPO-135M/commit/3502325a0dfd8894a42a544e02b16fb7f3548421', commit_message='Upload LlamaForCausalLM', commit_description='', oid='3502325a0dfd8894a42a544e02b16fb7f3548421', pr_url=None, repo_url=RepoUrl('https://huggingface.co/prxshetty/SmolGRPO-135M', endpoint='https://huggingface.co', repo_type='model', repo_id='prxshetty/SmolGRPO-135M'), pr_revision=None, pr_num=None)